In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel

In [2]:
!nvidia-smi

Fri May  6 19:46:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.15       Driver Version: 512.15       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:2B:00.0  On |                  N/A |
| 30%   44C    P8     5W / 170W |   1326MiB / 12288MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
torch.manual_seed(36)

### Loading GPT2-Medium Model from 🤗 Model Hub 

In [4]:

#indonesian-nlp/gpt2-medium-indonesian
model = GPT2LMHeadModel.from_pretrained('indonesian-nlp/gpt2-medium-indonesian').cuda()
tokenizer = GPT2Tokenizer.from_pretrained('indonesian-nlp/gpt2-medium-indonesian', bos_token='<|startoftext|>',
                                          eos_token='<|endoftext|>', pad_token='<|pad|>')

model.resize_token_embeddings(len(tokenizer))


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Embedding(50259, 1024)

In [5]:
captions = pd.read_csv('list_caption.csv')['processed']

In [6]:
max_length = max([len(tokenizer.encode(caption, max_length=512)) for caption in captions])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [7]:
class CaptionDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            encodings_dict = tokenizer('<|startoftext|>' + txt + '<|endoftext|>', truncation=True,
                                       max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [8]:
dataset = CaptionDataset(captions, tokenizer, max_length=max_length)
train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])


In [9]:
import gc
gc.collect()

13

In [10]:
torch.cuda.empty_cache()

In [11]:
training_args = TrainingArguments(output_dir='./results', num_train_epochs=2, logging_steps=100, save_steps=8000,
                                  per_device_train_batch_size=1, per_device_eval_batch_size=1,
                                  warmup_steps=10, weight_decay=0.05, logging_dir='./logs', report_to = 'none', fp16=True)


In [12]:
Trainer(model=model,  args=training_args, train_dataset=train_dataset, 
        eval_dataset=val_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])}).train()

Using amp half precision backend
D:\Anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 10428
  Num Epochs = 2
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 20856


Step,Training Loss
100,2.265700
200,0.747000
300,0.667000
400,0.668500
500,0.733800
600,0.642500
700,0.714000
800,0.638500
900,0.628000
1000,0.620300


Saving model checkpoint to ./results\checkpoint-8000
Configuration saved in ./results\checkpoint-8000\config.json
Model weights saved in ./results\checkpoint-8000\pytorch_model.bin
Saving model checkpoint to ./results\checkpoint-16000
Configuration saved in ./results\checkpoint-16000\config.json
Model weights saved in ./results\checkpoint-16000\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=20856, training_loss=0.5134400348348731, metrics={'train_runtime': 6423.4993, 'train_samples_per_second': 3.247, 'train_steps_per_second': 3.247, 'total_flos': 1.936898168507597e+16, 'train_loss': 0.5134400348348731, 'epoch': 2.0})

In [13]:
#save model
model.save_pretrained('./results')
tokenizer.save_pretrained('./results')

Configuration saved in ./results\config.json
Model weights saved in ./results\pytorch_model.bin
tokenizer config file saved in ./results\tokenizer_config.json
Special tokens file saved in ./results\special_tokens_map.json
added tokens file saved in ./results\added_tokens.json


('./results\\tokenizer_config.json',
 './results\\special_tokens_map.json',
 './results\\vocab.json',
 './results\\merges.txt',
 './results\\added_tokens.json')

### OA Line Caption Generator

In [157]:
generated = tokenizer("<|startoftext|> Pengumuman Pemenang", return_tensors="pt").input_ids.cuda()

In [158]:
sample_outputs = model.generate(generated, do_sample=True, top_k=50, 
                                max_length=512, top_p=1.5, temperature=1.2,num_return_sequences=10 )


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [159]:
for i, sample_output in enumerate(sample_outputs):
    print("%d."%(i+1),tokenizer.decode(sample_output, skip_special_tokens=True))

1.  Pengumuman Pemenang Lomba Esai dan Poster  -.
Selamat malam Mahasiswa Teknik Biomedis  
Kami, KMTM UGM mengucapkan terima kasih yang tak terkira atas antusiasme peserta dan dukungan pihak-pihak yang ikut berkontribusi dalam keseruan PPSMB 2019. 
Total terdapat 1020 peserta yang meramaikan setiap rangkaian kegiatan PPSMB UGM 2019. 
Semoga pengalaman dan kegiatan-kegiatan di PPSMB UGM 2019 memiliki kesan yang tak terlupakan sehingga ke depannya dapat memicu semangat teman-teman dalam berkarya. 
See you guys! 
Sekretaris Umum
KMTM FT-UGM
2.  Pengumuman Pemenang dan Penarikan Undian Lomba FILMI CENDEKIA 2021 - Selamat kepada @adhifarizqoh (FILMI #1) sebagai Juara 2 Lomba Karya Tulis Ilmiah:
Meraih Juara 2 tingkat Regional FILMI - FILMI Regional. Salam Cendekia!
Kepada seluruh delegasi yang telah terdaftar dan mengikuti perlombaan, kami ucapkan selamat dan semangat dalam mengikuti lomba yang akan datang. Ikuti kegiatan pengembangan soft skills demi mewujudkan prestasi.  Semangat semua! 